In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.8.0


In [2]:
from google.colab import files
uploaded_file = files.upload()

Saving dataset3.csv to dataset3.csv


In [3]:
dataset = pd.read_csv('dataset3.csv')
dataset.head()

,Dates,Hours,Category,X,Y,Label
0,31/03/2015 23:00:00,23,ASSAULT,106.779610,-6.250459,1
1,31/03/2015 22:30:00,22,VEHICLE THEFT,106.888504,-6.201302,1
2,31/03/2015 22:30:00,22,VEHICLE THEFT,106.854430,-6.251243,1
3,31/03/2015 21:58:00,21,ASSAULT,106.805830,-6.127431,1
4,31/03/2015 21:45:00,21,VEHICLE THEFT,106.842910,-6.228551,1


In [4]:
dataset.shape

(14402, 6)

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14402 entries, 0 to 14401
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Dates     14402 non-null  object 
 1   Hours     14402 non-null  int64  
 2   Category  14402 non-null  object 
 3   X         14402 non-null  float64
 4   Y         14402 non-null  float64
 5   Label     14402 non-null  int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 675.2+ KB


In [6]:
dataset.drop(['Dates', 'Category'], axis=1, inplace=True)

In [7]:
xs = dataset.iloc[:, 0:-1]
ys = dataset.iloc[:, -1]

In [8]:
dataset.describe()

,Hours,X,Y,Label
count,14402.000000,14402.000000,14402.000000,14402.000000
mean,11.436884,106.842759,-6.196510,0.175739
std,6.976909,0.056754,0.044963,0.380612
min,0.000000,106.723350,-6.285473,0.000000
25%,6.000000,106.798924,-6.239150,0.000000
50%,11.000000,106.840000,-6.200000,0.000000
75%,18.000000,106.900000,-6.160000,0.000000
max,23.000000,106.941390,-6.120000,1.000000


In [9]:
normalizer = tf.keras.layers.Normalization(axis=-1)

In [10]:
normalizer.adapt(np.array(xs))

In [11]:
print(normalizer.mean.numpy())
first = np.array(xs[:1])

with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer(first).numpy())

[[ 11.436889  106.84279    -6.1965103]]
First example: [[ 23.   106.78  -6.25]]

Normalized: [[ 1.66 -1.11 -1.2 ]]


In [12]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(32, activation='relu'),
      layers.Dense(2, activation='softmax')
  ])

  model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['Accuracy'])
  return model

In [13]:
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 3)                7         
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                128       
                                                                 
 dense_1 (Dense)             (None, 2)                 66        
                                                                 
Total params: 201
Trainable params: 194
Non-trainable params: 7
_________________________________________________________________


In [14]:
%%time
history = dnn_model.fit(xs, ys, verbose=1, validation_split=0.2, epochs=10)

Epoch 1/10
361/361 [==============================] - 1s 2ms/step - loss: 0.4849 - Accuracy: 0.7682 - val_loss: 0.1851 - val_Accuracy: 0.9962
Epoch 2/10
361/361 [==============================] - 1s 2ms/step - loss: 0.4429 - Accuracy: 0.7942 - val_loss: 0.1522 - val_Accuracy: 0.9913
Epoch 3/10
361/361 [==============================] - 1s 2ms/step - loss: 0.4360 - Accuracy: 0.8025 - val_loss: 0.1591 - val_Accuracy: 0.9885
Epoch 4/10
361/361 [==============================] - 1s 2ms/step - loss: 0.4316 - Accuracy: 0.8080 - val_loss: 0.1706 - val_Accuracy: 0.9885
Epoch 5/10
361/361 [==============================] - 1s 2ms/step - loss: 0.4281 - Accuracy: 0.8122 - val_loss: 0.1636 - val_Accuracy: 0.9868
Epoch 6/10
361/361 [==============================] - 1s 2ms/step - loss: 0.4243 - Accuracy: 0.8175 - val_loss: 0.1557 - val_Accuracy: 0.9889
Epoch 7/10
361/361 [==============================] - 1s 2ms/step - loss: 0.4214 - Accuracy: 0.8207 - val_loss: 0.1683 - val_Accuracy: 0.9820
Epoch 

In [20]:
input_data = [2, 106.815, -6.16388]

prediction = dnn_model.predict([input_data])

print(prediction.flatten()[1])
print(np.argmax(prediction))

0.1651418
0


In [ ]:
export_dir = 'saved_model/1'
tf.saved_model.save(dnn_model, export_dir)

INFO:tensorflow:Assets written to: saved_model/1/assets


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)

converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

In [ ]:
import pathlib

tflite_model_file = pathlib.Path('./model.tflite')
tflite_model_file.write_bytes(tflite_model)

2976

In [ ]:
dnn_model.save('my_model.h5')